# Fast Food Restaurant Analysis in Four Metropolitan Cities in India

Importing Libraries

In [1]:
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
import folium

Initilising Foursquare API 

In [2]:
CLIENT_ID = '1E0AOV3BCRV0ZGWDR5EAPWFO1ARCZCXCVYTYLIFNZLAEATII' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1E0AOV3BCRV0ZGWDR5EAPWFO1ARCZCXCVYTYLIFNZLAEATII
CLIENT_SECRET:EYVEW53WWRSYGNSY1ZD24K5DU21PIBTMAF5QLZOTYQZ1IA1F


Creating url for the 4 cities and getting results for each by using fast food specific category ID

In [3]:
LIMIT = 500 # Maximum is 100
cities = ["New Delhi", 'Kolkata', 'Mumbai', 'Chennai']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d16e941735") # FAST FOOD PLACE CATEGORY ID
    results[city] = requests.get(url).json()

Cleaning JSON file

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

<ipython-input-4-7134362d7262>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  venues = json_normalize(results[city]['response']['groups'][0]['items'])


The Foursquare API Only gives us the nearest 100 venues in the city. So let's first check out their densities by our eyes by mapping them

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of fast food places in {city} = ", results[city]['response']['totalResults'])
print("\nNote: These values only show Top 100")

Total number of fast food places in New Delhi =  94
Total number of fast food places in Kolkata =  25
Total number of fast food places in Mumbai =  117
Total number of fast food places in Chennai =  64

Note: These values only show Top 100


## Mapping Cities

### New Delhi

In [6]:
maps[cities[0]]

### Kolkata

In [7]:
maps[cities[1]]

###  Mumbai

In [8]:
maps[cities[2]]

### Chennai

In [9]:
maps[cities[3]]

We can see that Mumbai leads the fast food restaurant business between the four metropolitian cities followed by Chennai. 

However, Let's have a concrete measure of this density.I will get the mean location of the fast food places which should be near to most of them if they are really dense or far if not. 

Next I will take the average of the distance of the venues to the mean coordinates. 

In [10]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print("Mean Distance from Mean coordinates of", city, end=": ")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Mean Distance from Mean coordinates of New Delhi: 0.08119025938178852
Mean Distance from Mean coordinates of Kolkata: 0.05505771248775004
Mean Distance from Mean coordinates of Mumbai: 0.07124055332895064
Mean Distance from Mean coordinates of Chennai: 0.04486949902037954


# Mapping fast food places with dist from Mean Coordinates

## New Delhi

In [11]:
maps[cities[0]]

## Kolkata

In [12]:
maps[cities[1]]

## Mumbai

In [13]:
maps[cities[2]]

## Chennai

In [14]:
maps[cities[3]]

### Thus, to conclude, we see that although Mumbai has the most number of fastfood places, it is Chennai which has a higher density of them (or more restaurants closer to the mean coordinates).
### Further Kolkata has the least number of fast food places and could do with a few more of them.

This means that, future fast food restaurants could do by starting there new business in the less competetive environment of the city of Kolkata. 